In [17]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [5]:
df_train = pd.read_csv('./train.csv', 
                       dtype={
                           'question1': np.str,
                           'question2': np.str
                       })
df_train['test_id'] = -1
df_test = pd.read_csv('./test.csv', 
                      dtype={
                          'question1': np.str,
                          'question2': np.str
                      })
df_test['id'] = -1
df_test['qid1'] = -1
df_test['qid2'] = -1
df_test['is_duplicate'] = -1

df = pd.concat([df_train, df_test])
df['question1'] = df['question1'].fillna('')
df['question2'] = df['question2'].fillna('')
df['uid'] = np.arange(df.shape[0])
df = df.set_index(['uid'])
print df.dtypes
del(df_train, df_test)

id               int64
is_duplicate     int64
qid1             int64
qid2             int64
question1       object
question2       object
test_id          int64
dtype: object


In [10]:
df['len1'] = df['question1'].str.len()
df['len2'] = df['question2'].str.len()

In [14]:
df['is_first_word_equal'] = df.apply(
    lambda r: int(r['question1'].split(' ')[0] == r['question2'].split(' ')[0]), axis=1)

In [19]:
features = ['len1', 'len2', 'is_first_word_equal']

df[features].head()

,len1,len2,is_first_word_equal
uid,,,
0,66,57,1
1,51,88,1
2,73,59,1
3,50,65,0
4,76,39,1


In [20]:
ix_train = df['is_duplicate'] != -1
ix_test = df['is_duplicate'] == -1

In [24]:
model = LogisticRegression()
model = model.fit(df[features][ix_train], df['is_duplicate'][ix_train])

In [36]:
# вернет матрицу с двумя столбцами:
#  в первом вероятность is_duplicate == 0
#  во втором вероятность is_duplicate == 1
# нам нужен второй
prediction = model.predict_proba(df[features][ix_test])[:, 1]

In [37]:
prediction

array([ 0.27144738,  0.29238127,  0.50181426, ...,  0.28526016,
        0.23050182,  0.30357197])

In [38]:
print prediction.shape

(2345796,)


In [39]:
df.loc[ix_test].shape

(2345796, 10)

In [41]:
df_submit = df.loc[ix_test].copy()

In [42]:
df_submit.shape

(2345796, 10)

In [43]:
df_submit['is_duplicate'] = prediction

In [46]:
df_submit[['test_id', 'is_duplicate']].head()

,test_id,is_duplicate
uid,,
404290,0,0.271447
404291,1,0.292381
404292,2,0.501814
404293,3,0.409534
404294,4,0.559139


In [51]:
df_submit[['test_id', 'is_duplicate']].to_csv('./submit.csv', index=False)

In [53]:
!cat ./submit.csv | head -10

test_id,is_duplicate
0,0.271447380086
1,0.292381274418
2,0.501814261434
3,0.409534229702
4,0.559138543006
5,0.28511193636
6,0.497672622406
7,0.174943413231
8,0.495692767732
cat: stdout: Broken pipe


In [56]:
!gzip ./submit.csv

результат:
 - значение функции стоимости: 0.52411
 - место: 1641 из 2174